<div style="width:image width px; font-size:75%; text-align:right;">
    <img src="img/pexels-brandon-montrone-1179229.jpg" width="width" height="height" style="padding-bottom:0.2em;" />
    <figcaption>Photo by Brandon Montrone on Pexels</figcaption>
</div>

# Machine Learning 1

**Applied Programming - Summer term 2022 - FOM Hochschule für Oekonomie und Management - Cologne**

**Lecture 07 - May 07, 2022**

*Dennis Gluesenkamp*

## Table of contents
* [Dataset retrieval](#dataretrieval)
* [Preprocessing](#preprocessing)
    * [Imputation](#preprocessing_imputation)
    * [Categorical encoding](#preprocessing_encoding)
    * [Train-test-split](#preprocessing_traintestsplit)
    * [Scaling](#preprocessing_scaling)
* [Supporting techniques for modelling](#support)
    * [Cross validation](#support_cv)
    * [Grid search](#support_grid)
    * [Combined application in Scikit-learn](#support_combined)
* [Classification](#classification)
    * [Decision tree classifier](#classification_dt)
    * [Random forest classifier](#classification_rf)
    * [Homework](#classification_homework)
* [References](#references)

## Dataset retrieval<a class="anchor" id="dataretrieval"></a>
For the following work on machine learning algorithms, we need some data sets. These are generated or provided in a first step.

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_diabetes

%matplotlib inline

np.random.seed(42)

In [ ]:
# Artificial classification problem:
X_c, y_c = make_classification(n_samples            = 1000,
                               n_features           = 10,
                               n_redundant          = 2,
                               n_informative        = 5,
                               n_classes            = 3,
                               n_clusters_per_class = 3,
                               flip_y               = 0.05,
                               shift                = None,
                               random_state         = 42)

# Artificial regression problem:
X_r, y_r = make_regression(n_samples     = 1000,
                           n_features    = 10,
                           n_informative = 5,
                           noise         = 0.05,
                           random_state  = 42)

# Scikit-learn toy dataset for classification:
X_b, y_b = load_breast_cancer(return_X_y = True)

# Scikit-learn toy dataset for regression:
X_d, y_d = load_diabetes(return_X_y = True)

# Kaggle dataset for classification:
# Rain in Australia - Predict next-day rain in Australia
# https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
df_w = pd.read_csv('dat/weatherAUS.csv')

# Kaggle dataset for regression:
# Medical Cost Personal Datasets - Insurance Forecast by using Linear Regression
# https://www.kaggle.com/mirichoi0218/insurance
df_i = pd.read_csv('dat/insurance.csv')

## Preprocessing<a class="anchor" id="preprocessing"></a>
Every machine learning model creation starts with data preprocessing, where the raw data is converted into a format suitable for the later algorithm(s). This is necessary because real-world data is very often incomplete, inconsistent, unclean, or even contains errors. These shortcomings, which can hinder or prevent the creation of an algorithm, can be completely or partially eliminated by data preprocessing. Thus, preprocessing is a fundamental step that makes machine learning possible.
Preprocessing is composed of different steps, each with different objectives:

* The so-called **imputation** addresses existing missing values and replaces them, if possible, with suitable substitutions.
* Categorical variables are converted into a numerical value during **encoding**, which can be processed by the algorithm in contrast to the categories.
* To test the performance of the model, the dataset is split into two sets during the **training-test-split**.
* Some algorithms require numerical input in a certain scale range, which is produced by **scaling**.

Not every step is required for every algorithm or useful in every approach. This must be examined on a case-by-case basis.

### Imputation<a class="anchor" id="preprocessing_imputation"></a>
The artificially created data sets do not contain any missing values by definition respectively by construction. For the other data sets this has to be checked.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 8), sharey = False)
fig.suptitle('Inspection of missing values')

sns.heatmap(ax = axes[0, 0], data = pd.DataFrame(X_b).isnull(), cbar = False)
axes[0, 0].set_title('X of breast cancer dataset')
sns.heatmap(ax = axes[0, 1], data = pd.DataFrame(y_b).isnull(), cbar = False)
axes[0, 1].set_title('y of breast cancer dataset')

sns.heatmap(ax = axes[1, 0], data = pd.DataFrame(X_d).isnull(), cbar = False)
axes[1, 0].set_title('X of diabetes dataset')
sns.heatmap(ax = axes[1, 1], data = pd.DataFrame(y_d).isnull(), cbar = False)
axes[1, 1].set_title('y of diabetes dataset')

plt.draw()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 8), sharey = False)
fig.suptitle('Inspection of missing values')

sns.heatmap(ax = axes[0], data = df_w.isnull(), cbar = False)
axes[0].set_title('DataFrame of weather dataset')

sns.heatmap(ax = axes[1], data = df_i.isnull(), cbar = False)
axes[1].set_title('DataFrame of insurance dataset')

plt.draw()

In [ ]:
df_w.info()

There are quite a lot of missing values in the Australian weather dataset. This includes several attributes in varying degrees. We consider which imputation strategy to use on a case-by-case basis. From Scikit-learn, the ``SimpleImputer`` class is available for this purpose, which includes ``strategy`` as a parameter:
* ``mean``: Replace missing values with mean of the affected column, which has to be numeric.
* ``median``: Replace missing values with median of the affected column, which has to be numeric.
* ``most_frequent``: Replace missing with the most frequent value of the column (smallest value if there is more than one).
* ``constant``: Replace missing values with given ``fill_value``, which is also a parameter of the class.

However, the ``fillna()`` method from pandas can also be used here and is particularly useful for different procedures within a data set.

First, the numerical attributes are processed, for which a substitution based on statistical measures would be possible. Here we decide to replace with the mean value. These include:
* ``MinTemp``
* ``MaxTemp``
* ``Rainfall``
* ``WindGustSpeed``
* ``WindSpeed9am``
* ``WindSpeed3pm``
* ``Humidity9am``
* ``Humidity3pm``
* ``Pressure9am``
* ``Pressure3pm``
* ``Temp9am``
* ``Temp3pm``

*Comment: Whether a replacement with the mean value really makes sense should also be checked, for example, by looking at the distribution of the attribute. However, the scope at this point should not be exceeded by this exploration, which is necessary in reality, but too small-scale for the purposes of this lecture.*

In addition, there are four other numeric attributes, but they have a substantially higher proportion of missing values in comparison. The high percentage of almost 50% is a considerable obstacle. One possible strategy may be to exclude such sparsely populated attributes. Since we also want to include this approach here, the following four attributes are dropped without further examination.
* ``Evaporation``
* ``Sunshine``
* ``Cloud9am``
* ``Cloud3pm``

Finally, the categorical attributes are left. For the wind directions, a meaningful substitution is difficult. For this reason, the missing values are replaced here with a new category M. This also preserves the information about the missing, but the algorithm can be trained. This affects namely the following attributes:
* ``WindGustDir``
* ``WindDir9am``
* ``WindDir3pm``

For the categorical attribute ``RainToday``, we take a slightly different approach. We interpret a missing value at this point as if there had been no rain, which corresponds to the most frequent value.

It is interesting to note that the target variable ``RainTomorrow`` also includes missing values. This is of course a problem for supervised learning. There are methods to approximate the correct group, but we will take the faster approach and drop the affected objects.

In [ ]:
# Exclude rows with missing target
df_w = df_w[df_w['RainTomorrow'].notnull()]

In [ ]:
# Imputation with mean for numeric columns
num_imputation = ['MinTemp',
                  'MaxTemp',
                  'Rainfall',
                  'WindGustSpeed',
                  'WindSpeed9am',
                  'WindSpeed3pm',
                  'Humidity9am',
                  'Humidity3pm',
                  'Pressure9am',
                  'Pressure3pm',
                  'Temp9am',
                  'Temp3pm']
for c in num_imputation:
    df_w[c].fillna(df_w[c].mean(), inplace = True)

In [ ]:
# Exclude attributes with high percentage of missing values
df_w.drop(labels = ['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis = 1, inplace = True)

In [ ]:
# Imputation with 'M' for categorical wind directions
cat_imputation = ['WindGustDir', 'WindDir9am', 'WindDir3pm']
for c in cat_imputation:
    df_w[c].fillna('M', inplace = True)

In [ ]:
df_w.info()

In [ ]:
df_w.sample(5)

In [ ]:
from sklearn.impute import SimpleImputer
imp_rain = SimpleImputer(strategy = 'most_frequent')

#col_names = df_w.columns
#df_w = pd.DataFrame(imp_rain.fit_transform(df_w), columns = col_names).infer_objects()

In [ ]:
# Don't use dates in this example
df_w.drop(labels = ['Date'], axis = 1, inplace = True)

### Categorical encoding<a class="anchor" id="preprocessing_encoding"></a>
Many machine learning algorithms accept only numerical attributes as input, since they cannot map and process textual information, for example. Therefore, in such cases, encoding the categorical variables becomes necessary. However, care must be taken to avoid introducing additional and incorrect information into the data set. For example, it would not be appropriate to represent "female" and "male" by 1 and 2. This would generally be associated with a scale level and "male" would be considered twice as weighty. For other categorical variables, it may make sense to assign a scale or at least an order, such as clothing sizes, S, M, and L.

All categorical variables in our example have no such order. Therefore, we concentrate here on the so-called **"One Hot Encoding"**, which inserts a separate column for each expression of the attribute and thus only makes binary statements. For further methods like Ordinal Encoding or more complex approaches, please refer to the literature of your choice.

In [ ]:
print(df_w.info())
print(df_i.info())

In [ ]:
# One Hot Encodung with pandas' get_dummies
df_w_enc = pd.get_dummies(df_w, drop_first = True)
df_i_enc = pd.get_dummies(df_i, drop_first = True)

In [ ]:
df_w_enc.shape

In [ ]:
df_i_enc.info()

If there are attributes with a large number of different values, the One Hot Encoding can of course quickly lead to a very large number of additional columns. This is the case here in the weather dataset, where the number increases to over 3,500 columns.

### Train-test-split<a class="anchor" id="preprocessing_traintestsplit"></a>

The models created in the machine learning process are designed to learn relationships and patterns based on existing data so that they can apply and reproduce them on new, previously unknown data, in effect making a prediction. This property of the models to classify new data appropriately is called **generalization**. It follows that testing a model with training data is a serious methodological error. After all, this would only make the model fit the already known data particularly well. This (perhaps even perfect) result is called **overfitting**. To avoid this, a portion of the available data is set aside as a test set prior to the supervised learning procedure. It is important to make a randomized split here, which can be achieved in scikit-learn with the ``train_test_split`` class.

In [ ]:
from sklearn.model_selection import train_test_split

X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(X_c, y_c,
                                                            test_size = 0.25,
                                                            random_state = 42)

X_r_train, X_r_test, y_r_train, y_r_test = train_test_split(X_r, y_r,
                                                            test_size = 0.25,
                                                            random_state = 42)

X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(X_b, y_b,
                                                            test_size = 0.3,
                                                            random_state = 42)

X_d_train, X_d_test, y_d_train, y_d_test = train_test_split(X_d, y_d,
                                                            test_size = 0.3,
                                                            random_state = 42)

In [ ]:
print(X_c.shape)
print(y_c.shape)
print(X_c_train.shape)
print(y_c_train.shape)

In [ ]:
df_w_train, df_w_test = train_test_split(df_w_enc,
                                         test_size = 0.35,
                                         random_state = 42)

df_i_train, df_i_test = train_test_split(df_i_enc,
                                         test_size = 0.35,
                                         random_state = 42)

### Scaling<a class="anchor" id="preprocessing_scaling"></a>
Variables can include values in very different numerical ranges. Restricting this range so that the attributes can be treated on the same basis or weighting is called scaling.

For example, looking at the health care cost dataset, we see that age and number of children can take on substantially different magnitudes from each other, i.e., they do not have the same scaling.

In [ ]:
df_i.sample(10)

Such scale differences are also found in the Australian weather data set. Here, especially the air pressure would be overweighted.

In [ ]:
df_w.select_dtypes(include = ['float64'])

This usually leads to erroneous conclusions in many machine learning models (SVM, neural networks, etc.), since most are based on the Euclidean distance measure resp. gradient descent. Thus, in the example, the age would clearly dominate the number of children, just by the fact that the numerical value is larger. This has nothing to do with the underlying mechanisms that are supposed to be trained.

For tree-based algorithms (decision tree, random forest, etc.), the situation is somewhat different. These algorithms determine a split of the data based only on the properties of an attribute. This is to increase the homogeneity in a node. However, this also means that other features and thus their scale level do not influence this split. Tree-based algorithms are therefore invariant to different scales.

In the context of data analysis, **normalization** and **standardization** are particularly relevant as scaling techniques. Normalization means that the data is scaled to the closed interval [0, 1]. With standardization, the data is transformed to mean value zero and standard deviation one. Now, which technique is used under which circumstances? There is no general answer to this question. However, some hints can be given that can be considered when making a choice. If the data certainly do not follow a normal distribution, then the obvious choice is to use normalization. In particular, this is useful for modeling that does not assume a particular distribution anyway, such as neural networks. Standardization, on the other hand, can be applied if the data follow a normal distribution, but this is not necessarily the case. The important difference is that standardization, unlike normalization, does not limit the scale of results to a closed interval. This means that the characteristic of outliers remains.

For normalization, scikit-learn offers the class ``MinMaxScaler()``, whose default parameters are already set to the interval [0, 1]. However, this interval can also be adjusted to individual intervals such as [-1, 1]. For standardization, the class ``StandardScaler()`` transforms the data to mean value zero and variance one.

After superficial review, we will use normalization for the weather dataset and standardization for the insurance costs. We make this simplification due to time constraints of the lecture and it is not intended to claim to be the perfectly correct choice.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
df_w_train_scaled = pd.DataFrame(mm_scaler.fit_transform(df_w_train),
                                 columns = df_w_train.columns)
df_w_test_scaled  = pd.DataFrame(mm_scaler.transform(df_w_test),
                                 columns = df_w_test.columns)

In [ ]:
df_w_train_scaled.describe()

In [ ]:
df_w_test_scaled.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
st_scaler = StandardScaler()
df_i_train_scaled = pd.DataFrame(st_scaler.fit_transform(df_i_train),
                                 columns = df_i_train.columns)
df_i_test_scaled  = pd.DataFrame(st_scaler.transform(df_i_test),
                                 columns = df_i_test.columns)

In [ ]:
df_i_train_scaled.describe()

In [ ]:
df_i_test_scaled.describe()

At this point, we would like to make a remark about the order of preprocessing: We first divided the data into a training and a test set and then scaled the data, namely only on the training data. This avoids the so-called **data leakage**. Data leakage means that information that does not belong to the training data set is included in the training. This would be the case if we had first performed the scaling and then the split, since the distribution information of the test set would then have been taken into account during the scaling. This must be ruled out under all circumstances.

## Supporting techniques for modelling<a class="anchor" id="support"></a>

### Cross validation<a class="anchor" id="support_cv"></a>

To get a reliable value for the goodness of fit of a model, we have already performed the split in training and test data. This is to prevent overfitting by considering the error on the test data (*out-of-sample error*; in contrast to error with training data *in-sample error*). However, overfitting can also be counteracted during the training process by the **cross-validation** procedure.

In k-fold cross-validation, the training data set, which consists of $n$ objects, is divided into $k \leq n$ (as close as possible) equally sized partial data sets. Training is now always performed on $k-1$ subsets and tested against the remaining subset. The total error is calculated as the average of the individual errors. The working schema is shown in the following figure created by Scikit-learn developers.

<div style="width:image width px; font-size:75%; text-align:center;">
    <img src="img/grid_search_cross_validation.png" width=500 style="padding-bottom:0.2em;" />
    <figcaption>Schema of k-fold cross-validation [1]</figcaption>
</div>

### Grid search<a class="anchor" id="support_grid"></a>
Accurate parameter tuning of the modeling is also an important aspect for improving the performance of a model. The search for the most favorable combination is called hyperparameter optimization. Hyperparameters are the parameters that must be set before training the model and control the behavior of the training algorithm. A comprehensive approach to hyperparameter optimization is to test through all possible combinations of settings. This procedure is called **grid search**. In addition, there is a whole range of sophisticated techniques, but they are beyond the scope of this lecture.

### Combined application in Scikit-learn<a class="anchor" id="support_combined"></a>

The ``sklearn.model_selection.GridSearchCV`` class within Scikit-learn provides the combined use of cross validation and grid search, keeping the process particularly straightforward. Let's look in details at the following three parameters:
* ``estimator``: Instance of the classifier resp. regressor, e.g. ``DecisionTreeClassifier()``
* ``param_grid``: Dictionary of hyperparameters and the associated values, e.g. ``{'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}``
* ``cv``: Number of folds $k$ for cross validation

In [ ]:
from sklearn.model_selection import GridSearchCV

## Classification<a class="anchor" id="classification"></a>
Now that we have completed the preprocessing and learned about other auxiliary functionalities, we begin with the actual machine learning models, looking at two parts in particular: classification and regression.

In the first part, we consider classification problems, that is, the attempt to assign an object to a defined and delimited group. These groups can be represented by digits, numbers, categories, or even Boolean expressions. In principle, the number of categories is open upwards, although usually and often in real problems the number is strongly limited (one digit). The following figures show the plots of the randomly generated classification dataset and the Australian weather, with the various classes colored differently.

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(X_c[:, 0], X_c[:, 4], s = 25, c = y_c, alpha = 0.6)
plt.xlabel('X_0')
plt.ylabel('X_4')
plt.title('Artificial classification problem')
plt.draw()

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(df_w_train_scaled['MaxTemp'], df_w_train_scaled['MinTemp'],
            s = 25, c = df_w_train_scaled['RainTomorrow_Yes'], alpha = 0.3)
plt.xlabel('Maximum Temperature')
plt.ylabel('Minimum temperature')
plt.title('Australian weather classification problem')
plt.draw()

### Decision tree classifier<a class="anchor" id="classification_dt"></a>
Decision trees are directed trees where the nodes represent a decision between different instances of an attribute. The edges following this node are the respective decision for one of the possibilities. The nodes of the next layer following the edges finally represent the classification respectively the decision made. This is how the decision tree divides the state space of the problem into small subsets. Each level of the decision tree
allows the state space to be further fragmented. In this way, a decision tree forms a set of decision rules that is used as a classification technique. The following figure illustrates this process with an example.

<div style="width:image width px; font-size:75%; text-align:center;">
    <img src="img/fig_DecisionTree.png" width=800 style="padding-bottom:0.2em;" />
    <figcaption>Decision process of decision tree classifier</figcaption>
</div>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix

dt     = DecisionTreeClassifier(random_state = 42)
params = {'criterion':    ['gini', 'entropy'],
          'splitter':     ['best', 'random'],
          'max_depth':    [2, 3, 4, 5, 6],
          'class_weight': ['balanced', None]}
k      = 3

clf = GridSearchCV(dt, params, cv = k)
clf.fit(X_c_train, y_c_train)

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values(by = ['rank_test_score'])

In [ ]:
y_c_pred = clf.predict(X_c_test)
print(accuracy_score(y_c_test, y_c_pred))

In [ ]:
plot_confusion_matrix(clf, X_c_test, y_c_test)  
plt.draw()

In [ ]:
from sklearn import tree
plt.figure(figsize = (75, 8))
tree.plot_tree(clf.best_estimator_)
plt.show()

#### Exercise
Take the scheme shown above for a decision tree classifier with the sections:
* Instantiation of the classifier
* Definition of the hyperparameter space
* Defining the number of folds for CV
* Fitting the classifier
* Calculation of the accuracy
* Plot the confusion matrix

*Optional:* If you like, add more hyperparameters of the decision tree to the grid search and/or change the number of folds.

Train one model for the breast cancer and one model for the rainy weather dataset. Try to achieve as high an accuracy as possible.

### Random forest classifier<a class="anchor" id="classification_rf"></a>
A single decision tree can have a high variance and error rate because of its large number of decision nodes and layers. This is due to the uncertainty that each node entails. To improve the predictive power an ensemble of decision trees, called random forest, was proposed by Leo Breiman [[2]](#breiman) as a classification method, however, they can also be used for regression. Which class is predicted depends on the selected technique. Breiman’s algorithm chose the class most frequently mentioned by the trees of the forest. An alternative is to take into account the probabilities that existed when the path reached the leaf of the tree. This
probabilistic prediction can be used via the mean value to forecast the class with a certain
probability.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf     = RandomForestClassifier(random_state = 42)
params = {'n_estimators': [100, 200],
          'criterion':    ['gini', 'entropy'],
          'max_depth':    [10, 12, 14, 16],
          'class_weight': ['balanced', None]}
k      = 3

clf = GridSearchCV(rf, params, cv = k)
clf.fit(X_c_train, y_c_train)

y_c_pred = clf.predict(X_c_test)
print(accuracy_score(y_c_test, y_c_pred))

plot_confusion_matrix(clf, X_c_test, y_c_test)  
plt.draw()

In [ ]:
pd.DataFrame(clf.cv_results_).sort_values(by = ['rank_test_score'])

#### Exercises
Train one random forest model for the breast cancer and one model for the rainy weather dataset. Try to achieve as high an accuracy as possible - at least better than the single decision tree.

### Homework<a class="anchor" id="classification_homework"></a>
Pick another alternative classification algorithm available at Scikit-learn. Try to achieve an even better result (accuracy) with this one for the breast cancer and weather dataset. Be open to experimentation!

## References<a class="anchor" id="references"></a>

[1]<a class="anchor" id="sklearn2021a"></a> The scikit-learn developers (2021). 3.1. Cross-validation: evaluating estimator performance. Retrieved 2021-05-15 from https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

[2]<a class="anchor" id="breiman"></a> Leo Breiman. "Random forests". In: Machine learning 45.1 (2001), pp. 5–32.